# These are routines to put in physically motivated profiles --- either that match the X-ray data or simulations

In [95]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from scipy.interpolate import interp1d
import scipy.special as sc
import scipy.integrate as integrate

In [96]:
#fundamental constants
mp = 1.6726216e-24  #proton mass in gr
KPCTOCM = 3.086e21
Msun =  1.9889e33  # gr 

#cosmology "constants"
Omegab = 0.045
OmegaM=0.3 
fb = Omegab/OmegaM #fraction of matter in baryons
h = 0.7
rhocrit = 2.7755e11*h*h
YHE = 0.25 #helium mass fraction
mu = 1/((1-YHE) + 2.*YHE/4)  #mean molecular weight to give number of electrons assuming helium doubly ionized


In [97]:
#from memory.  Adnan, substitute your expression 
def rvir(M, z):
    return 280*(M/1e12)**(.33)/(1+z) #kpc

#radius that is 200 times the matter density in kpc  (very similar to rvir)
def r200Mz(Mhalo, z):
    rhomatter = rhocrit*OmegaM
    return (Mhalo/((4*np.pi/3)*200.*rhomatter))**(1./3.)*1e3;

# I'm not sure I'm using this
def rhoNFW(r, rs):
    return 1/(r/rs)/(1+(r/rs))**2

# Voit Perciptation limited model from Appendix A in  https://arxiv.org/pdf/1811.04976.pdf

In [388]:
from scipy.interpolate import interp1d
from scipy.integrate import quad


def nePrecipitation(logMhalo,Rvirkpc,resolution,redshift,XRvir=2, Zmetal=0.3, tratcrit=10):
    
    #Zmetal is the metalicity; tratcrit is the coolin crieteria -- both of these don't need to change
    # XRvir is how many virial radii to go out
    fbaryon=0.2 #put in correct value
#     Rvirkpc = 300 #put in correct value here
    
    fitarray = np.array([[350, 8e12, 10, 0.5, 2.7,  0.73, 1.2e-1, 1.2, 3.8e-4,  2.1], \
          [350, 8e12, 10, 0.3, 2.4,  0.74, 1.5e-1, 1.2, 4.2e-4, 2.1], \
    # [350, 8e12, 10 &  $Z_{\rm grad}$ & 3.6  &  0.68 &  $8.6 \times 10^{-2}$ & 1.1 & $4.1 \times 10^{-4}$ & 2.0 \\
    # [300, 5.1e12,  & 10 &  1.0 & 3.3  &  0.71 & 5.6 \times 10^{-2}$  & 1.2 & $1.7 \times 10^{-4}$ & 2.1 \\
          [300, 5.1e12,  10,  0.5, 2.6,  0.72, 8.0e-2, 1.2, 2.3e-4,  2.1], \
          [300, 5.1e12,  10,  0.3, 2.3, 0.73,  9.6e-2, 1.2, 2.6e-4,  2.1], \
    #300 &  $5.1 \times 10^{12}$  & 10 &  $Z_{\rm grad}$ & 3.4  &  0.67 &  $5.8 \times 10^{-2}$ & 1.1 & $2.6 \times 10^{-4}$ & 2.1 \\
    #[300, 5.1e12,  20, 1.0,  5.2, 0.70, 2.9e-2, 1.1, 1.0e-4, 2.1],
          [300, 5.1e12, 20, 0.5,  4.1,  0.71, 4.3e-2, 1.1, 1.4e-4, 2.1], \
          [300, 5.1e12, 20, 0.3, 3.6,  0.71, 5.1e-2, 1.2, 1.6e-4, 2.1], \
    #300 &  $5.1 \times 10^{12}$  & 20 &  $Z_{\rm grad}$ & 5.4  &  0.65 &  $3.0 \times 10^{-3}$ & 1.1 & $1.6 \times 10^{-4}$ & 2.1 \\
    # 250 &  $2.9 \times 10^{12}$  & 10 &  1.0 & 3.7  &  0.71 &  $2.8 \times 10^{-2}$ & 1.2 & $8.1 \times 10^{-5}$ & 2.2 \\
          [250, 2.9e12, 10,  0.5, 2.8, 0.72, 4.2e-2, 1.2, 1.1e-4, 2.2], \
          [250, 2.9e12, 10,  0.3, 2.4,  0.72, 5.1e-3, 1.2, 1.3e-4, 2.2], \
    #250 &  $2.9 \times 10^{12}$  & 10 &  $Z_{\rm grad}$ & 3.7 &  0.66  &  $2.9 \times 10^{-2}$ & 1.1 & $1.3 \times 10^{-4}$ & 2.1 \\
    #220 &  $2.0 \times 10^{12}$  & 10 &  1.0 & 4.0  &  0.70    &  $1.7 \times 10^{-2}$ & 1.2 & $4.2 \times 10^{-5}$ & 2.3 \\
         [220, 2.0e12,  10,  0.5, 3.0,  0.71, 2.5e-2, 1.2, 6.1e-5, 2.2], \
         [220, 2.0e12,  10, 0.3, 2.6,  0.71, 3.1e-2, 1.2, 7.2e-5, 2.2], \
    #220 &  $2.0 \times 10^{12}$  & 10 &  $Z_{\rm grad}$ & 4.1  &  0.65 &  $1.8 \times 10^{-2}$ & 1.1 & $7.4 \times 10^{-5}$ & 2.2 \\
    #220 &  $2.0 \times 10^{12}$  & 20 &  1.0 & 6.3  &  0.69  &  $8.6 \times 10^{-3}$ & 1.1 & $2.3 \times 10^{-5}$ & 2.3 \\
         [220, 2.0e12,  20,  0.5, 4.8, 0.70, 1.3e-2, 1.1, 3.4e-5, 2.2], \
         [220, 2.0e12,  20,  0.3, 4.1,  0.70, 1.6e-2, 1.1, 4.2e-5, 2.2], \
    #220 &  $2.0 \times 10^{12}$  & 20 &  $Z_{\rm grad}$ & 6.5  &  0.63 &  $9.2 \times 10^{-3}$ & 1.1 & $4.3 \times 10^{-4}$ & 2.1 \\
    # 180 &  $1.1 \times 10^{12}$  & 10 &  1.0 & 5.6  &  0.71  &  $5.4 \times 10^{-3}$ & 1.2 & $9.6 \times 10^{-6}$ & 2.2 \\
         [180, 1.1e12, 10,  0.5, 4.0,  0.71, 8.7e-3, 1.2, 1.6e-5, 2.2], \
         [180, 1.1e12, 10,  0.3, 3.4,  0.71, 1.1e-2, 1.2, 2.1e-5, 2.2], \
    #180 &  $1.1 \times 10^{12}$  & 10 &  $Z_{\rm grad}$ & 5.7  &  0.63 &  $5.9 \times 10^{-3}$ & 1.1 & $2.3 \times 10^{-5}$ & 2.2 \\
         [150, 6.3e11, 10,  0.5, 6.1,  0.68, 2.8e-3, 1.2, 7.4e-6, 2.3], \
         [150, 6.3e11, 10, 0.3, 4.9,  0.68, 3.4e-3, 1.1, 9.8e-6, 2.2], \
         [150, 6.3e11, 10, 0.1, 3.0, 0.69, 8.1e-3, 1.2, 1.9e-5, 2.2], \
         [120, 3.2e11, 10, 0.5, 6.1,  0.69, 1.4e-3, 1.2, 2.3e-6, 2.2], \
         [120, 3.2e11, 10, 0.3, 5.0,  0.70, 1.9e-3, 1.2, 2.9e-6, 2.2], \
         [120, 3.2e11, 10, 0.1, 3.1,  0.71, 3.7e-3, 1.2, 5.1e-6, 2.2], \
         [120, 3.2e11, 20, 0.5, 9.6,  0.69, 7.0e-4, 1.2, 1.2e-6, 2.2],\
         [120, 3.2e11, 20, 0.3, 7.9,  0.70, 9.5e-4, 1.2, 1.5e-6, 2.2],\
         [120, 3.2e11, 20, 0.1, 4.9,  0.71, 1.9e-3, 1.2, 2.7e-6, 2.2]])


    #chooses metalicity and cooling criterion parameters we will interpolate on
    reducedarr = fitarray[(fitarray[:, 3] == Zmetal) & (fitarray[:, 2] ==  tratcrit)]
    reducedarr = reducedarr[::-1] #reverses array
    
    #old interpolation
    #n1 = np.interp(logMhalo, np.log10(reducedarr[:, 1]), reducedarr[:, 6])
    #xi1 = np.interp(logMhalo, np.log10(reducedarr[:, 1]), reducedarr[:, 7])
    #n2 = np.interp(logMhalo, np.log10(reducedarr[:, 1]), reducedarr[:, 8])
    #xi2 = np.interp(logMhalo, np.log10(reducedarr[:, 1]), reducedarr[:, 9])                  
    logMhalo_zp2 = logMhalo*((1+0.2)/(1+redshift))**(3/2)
    #better interpolation
    logn1 = interp1d(np.log10(reducedarr[:, 1]), np.log10(reducedarr[:, 6]), kind='linear', fill_value='extrapolate')(logMhalo_zp2)   
    n1 = 10**logn1
    xi1 = interp1d(np.log10(reducedarr[:, 1]), reducedarr[:, 7], kind='linear', fill_value='extrapolate')(logMhalo_zp2) 
    logn2 = interp1d(np.log10(reducedarr[:, 1]), np.log10(reducedarr[:, 8]), kind='linear', fill_value='extrapolate')(logMhalo_zp2)   
    n2 = 10**logn2
    xi2 = interp1d(np.log10(reducedarr[:, 1]), reducedarr[:, 9], kind='linear', fill_value='extrapolate')(logMhalo_zp2) 
        
    #print(logMhalo, n1, xi1, n2, xi2)

    rkpc = np.logspace(0, np.log10(Rvirkpc*XRvir), 300)#number of radial bins

    #Voit 2018 fitting formulae
    rhoarr = np.array([rkpc, 1/np.sqrt(1/(n1*(rkpc)**-xi1+ 1e-20)**2 + 1/(n2*(rkpc/100)**-xi2 + 1e-20)**2)])
    
    #Integrate to see how much mass is missed by this profile  (I've checked these seems reasonable)
#     rhointerp = interp1d(np.log(rhoarr[0]), 4.*np.pi*rhoarr[0]**3*rhoarr[1], kind='cubic', fill_value='extrapolate')
    rhointerp = interp1d(np.log(rhoarr[0]), 4.*np.pi*rhoarr[0]**3*rhoarr[1], kind='linear', fill_value='extrapolate')
#     conv = (1.989e33/(1.67e-24))/(3.086e21)**3  #constants: use your values, should have mean molecular weight, which is 1.2 
    conv = (Msun/(mu*mp))/KPCTOCM**3
    mtotal = quad(rhointerp, 0, np.log(XRvir*Rvirkpc))[0]/conv
    
    
    
    #add in rest of mass 
    neconstant =(10**logMhalo*fbaryon-mtotal)/(4.*np.pi*(XRvir*Rvirkpc)**3)*conv
#     print(neconstant)
    length = len(rkpc)
#     print("shape = ", length)
    for i in range(length):
        if rhoarr[0,  i] < XRvir*Rvirkpc:
#             rhoarr[1, i] = np.array([1/np.sqrt(1/(n1*(rkpc)**-xi1+ 1e-20)**2 + 1/(n2*(rkpc/100)**-xi2 + 1e-20)**2)])[1,i] + neconstant
            rhoarr[1, i] = rhoarr[1, i] + neconstant
    
    #print("ftotal =", mtotal/10**logMhalo/fbaryon, neconstant) #.2 is fraction of baryons
#     resolution=1
    h=.7
    L=250/h #Mpc
    res=1024*resolution
    cellsize = L*1000/res # kpc
    
#     x = np.ogrid[-10*resolution: 10*resolution]
    y,x = np.ogrid[-20*resolution: 20*resolution, -20*resolution:20*resolution]

#     f1= lambda x, y, z: my_func(((x**2+y**2+z**2)**.5), n1,n2,xi1,xi2,neconstant,cellsize)
    f1= lambda x, y, z: my_func(((x**2+y**2+z**2)**.5), n1,n2,xi1,xi2,neconstant,cellsize,Rvirkpc,XRvir,redshift)
    
    
    vec_integral=np.vectorize(func3Dto2D)
    
    

#     mask1 = vec_integral(f1,x,y,XRvir*Rvirkpc/cellsize)
    mask1 = vec_integral(f1,x,y,XRvir*Rvirkpc/cellsize)
    r=(x**2+y**2)**.5 # * scale_down
    mask1=mask1.astype(float)
    mask1[r > (XRvir*Rvirkpc/cellsize)]=0         

#     mask1[r <= (XRvir*Rvirkpc/cellsize)] =+ neconstant 
    
    
    return rhoarr,mask1

# nePercipitation(10)

# pass rkpc 

In [384]:
# The user can create their own function
# All length scales have to be converted into units of cellsize

def my_func(r, n1,n2,xi1,xi2,neconstant,cellsize_kpc,Rvirkpc,XRvir,redshift):
    
    
#     final_ar = np.array([1/np.sqrt(1/(n1*((r+.5)*cellsize_kpc)**-xi1)**2 + 1/(n2*((r+.5)*cellsize_kpc/100)**-xi2)**2)])# + neconstant
    x = (np.array(r) <= XRvir*Rvirkpc)


    final_ar = np.array([1/np.sqrt(1/(n1*((r+.5)*cellsize_kpc/(1+redshift))**-xi1)**2 + 1/(n2*((r+.5)*cellsize_kpc/(100*(1+redshift))**-xi2)**2)) + x.astype(int)*neconstant])
    
    

    return final_ar
           


def func3Dto2D(f,x,y,Rvir):
    
    return integrate.quad(f,-1*np.real((Rvir**2-(x**2+y**2))**.5),np.real((Rvir**2-(x**2+y**2))**.5),args=(x,y))[0] 
    


In [390]:
def func3Dto2D(f,x,y,Rvir):
     boundary = math.sqrt(max(0.0, Rvir**2-(x**2+y**2)))
    if boundary == 0.0:
        return 0.0
    else:
        #print("Boundary: {}".format(boundary))
        return 2 * integrate.quad(f, 0, boundary, args=(x,y))[0] 

    


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 3)

In [383]:
def radial_profile(data, center_x,center_y):
    y, x = np.indices((data.shape))
    r = np.sqrt((x - center_x)**2 + (y - center_y)**2)
    r = r.astype(np.int)

    tbin = np.bincount(r.ravel(), data.ravel())
    nr = np.bincount(r.ravel())
    radialprofile = tbin / nr
    return radialprofile 

# The user can create their own function
# All length scales have to be converted into units of cellsize

def my_func(r, n1,n2,xi1,xi2,neconstant,cellsize_kpc,Rvirkpc,XRvir):
    
    rkpc = np.logspace(0, np.log10(Rvirkpc*XRvir), 300)
    final_ar = np.array([rkpc, (1/np.sqrt(1/(n1*((r+.5)*cellsize_kpc)**-xi1)**2 + 1/(n2*((r+.5)*cellsize_kpc/100)**-xi2)**2))])# + neconstant
    
    length = len(rkpc)
#     print("shape = ", length)
    for i in range(length):
        if final_ar[0,  i] < XRvir*Rvirkpc:
#             rhoarr[1, i] = np.array([1/np.sqrt(1/(n1*(rkpc)**-xi1+ 1e-20)**2 + 1/(n2*(rkpc/100)**-xi2 + 1e-20)**2)])[1,i] + neconstant
            final_ar[1, i] = rhoarr[1, i] + neconstant
    
    return final_ar[1]
           


test = nePercipitation(14,600,1)

plt.imshow((test[1]))
plt.colorbar()

plt.imshow((test[1]))
plt.colorbar()

test2 = nePercipitation(14,1000,1)

test3 = nePercipitation(10,50,1)

plt.imshow((test[1]))
plt.colorbar()

test_rad= radial_profile(test[1], 20,20)

plt.plot(test_rad)

test2_rad= radial_profile(test2[1], 20,20)

plt.plot(test2_rad)

    h=.7
L=250/h #Mpc
res=1024
cellsize = L*1000/res # kpc
    

cellsize

m14 = nePercipitation(14)
print(m14[0])

m14 = nePercipitation(14)
print(m14[1])
m13 = nePercipitation(13)
m12 = nePercipitation(12)
m11 = nePercipitation(11)
m10 = nePercipitation(10)
plt.loglog(m14[0], m14[1])
plt.loglog(m13[0], m13[1])
plt.loglog(m12[0], m12[1])
plt.loglog(m11[0], m11[1])
plt.loglog(m10[0], m10[1])

#What the profile looks like making differnet assumptions
m13 = nePercipitation(13)
m12 = nePercipitation(12)
m13p5 = nePercipitation(13., .5)
m12p5 = nePercipitation(12, .5)
m1320 = nePercipitation(13, .3, 20)
m1220 = nePercipitation(12, .3, 20)

plt.loglog(m13[0], m13[1])
plt.loglog(m12[0], m12[1])

plt.loglog(m13p5[0], m13p5[1])
plt.loglog(m12p5[0], m12p5[1])

plt.loglog(m1320[0], m1320[1])
plt.loglog(m1220[0], m1220[1])